In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from linearmodels import PanelOLS, RandomEffects
from linearmodels.panel import compare
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('stock_daily_sentiment.csv')
df['date'] = pd.to_datetime(df['date'])
df['stock_code'] = df['stock_code'].astype(str).str.zfill(6)

df = df.set_index(['stock_code', 'date'])

In [3]:
All_Y = df[['open','close','high','low','volume','amount','amplitude','pct_change',
               'price_change','turnover_rate','forward_ret_1d','forward_ret_3d','forward_ret_5d']]
All_X = df[['avg_sentiment','sentiment_std','positive_ratio','negative_ratio',
            'avg_intensity','comment_count','avg_positive_prob','avg_negative_prob',
            'sentiment_net','sentiment_consensus','ma_3d','std_3d',
            'sentiment_change_3d','ma_5d','std_5d','sentiment_change_5d',
            'ma_10d','std_10d','sentiment_change_10d']]
Sel_Y = df[['close','volume','amount','amplitude','pct_change',
               'price_change','turnover_rate','forward_ret_1d','forward_ret_3d','forward_ret_5d']]
Sel_X = df[['avg_sentiment'#,'positive_ratio'
               ,'sentiment_std',
            'avg_intensity','comment_count',
            'sentiment_consensus','ma_3d','std_3d',
            'sentiment_change_3d']]

# 计算相关性矩阵
corr_matrix = All_X.corr()

# 找出高度相关的变量对(相关系数>0.8)
high_corr = np.where(np.abs(corr_matrix) > 0.8)
high_corr = [(corr_matrix.index[x], corr_matrix.columns[y], corr_matrix.iloc[x, y]) 
             for x, y in zip(*high_corr) if x != y and x < y]

# 查看高相关性的变量对
for var1, var2, corr in high_corr:
    print(f"{var1} - {var2}: {corr:.3f}")

avg_sentiment - positive_ratio: 0.992
avg_sentiment - negative_ratio: -0.992
avg_sentiment - avg_positive_prob: 1.000
avg_sentiment - avg_negative_prob: -1.000
avg_sentiment - sentiment_net: 0.992
positive_ratio - negative_ratio: -1.000
positive_ratio - avg_positive_prob: 0.992
positive_ratio - avg_negative_prob: -0.992
positive_ratio - sentiment_net: 1.000
negative_ratio - avg_positive_prob: -0.992
negative_ratio - avg_negative_prob: 0.992
negative_ratio - sentiment_net: -1.000
avg_positive_prob - avg_negative_prob: -1.000
avg_positive_prob - sentiment_net: 0.992
avg_negative_prob - sentiment_net: -0.992
ma_3d - ma_5d: 0.929
ma_3d - ma_10d: 0.866
std_3d - std_5d: 0.816
ma_5d - ma_10d: 0.940
std_5d - std_10d: 0.884


In [9]:
y1 = df['forward_ret_1d']
X1 = df[['avg_sentiment','sentiment_std','comment_count',
            'sentiment_consensus','ma_3d','std_3d']]
            
res1 = PanelOLS(y1, sm.add_constant(X1), entity_effects=True, time_effects=True).fit()
print(res1)

                          PanelOLS Estimation Summary                           
Dep. Variable:         forward_ret_1d   R-squared:                        0.0124
Estimator:                   PanelOLS   R-squared (Between):             -0.3090
No. Observations:                1879   R-squared (Within):               0.0080
Date:                Sun, Mar 16 2025   R-squared (Overall):             -0.0074
Time:                        15:30:31   Log-likelihood                    4501.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3.6571
Entities:                         111   P-value                           0.0013
Avg Obs:                       16.928   Distribution:                  F(6,1746)
Min Obs:                       11.000                                           
Max Obs:                       17.000   F-statistic (robust):             3.6571
                            

In [15]:
# 定义因变量列表
y_vars = ['close','volume','amount','amplitude','pct_change',
          'price_change','turnover_rate','forward_ret_1d',
          'forward_ret_3d','forward_ret_5d']

# 定义自变量
X_vars = ['avg_sentiment','sentiment_std','comment_count',
            'sentiment_consensus','ma_5d','std_5d']

# 存储结果的字典
regression_results = {}

# 批量回归
for y_var in y_vars:
    # 进行回归
    y = df[y_var]
    X = df[X_vars]
    model = PanelOLS(y, sm.add_constant(X), 
                     entity_effects=True, 
                     time_effects=True)
    results = model.fit()
    
    # 存储结果
    regression_results[y_var] = {
        'R-squared': results.rsquared,
        'R-squared Within': results.rsquared_within,
        'R-squared Between': results.rsquared_between,
        'R-squared Overall': results.rsquared_overall,
        'N_obs': results.nobs,
        'Parameters': results.params,
        't-stats': results.tstats,
        'P-values': results.pvalues,
        'F-statistic': results.f_statistic.stat
    }

# 打印结果摘要
for y_var, results in regression_results.items():
    print(f"\n=== 因变量: {y_var} ===")
    print(f"R-squared: {results['R-squared']:.4f}")
    print(f"R-squared Within: {results['R-squared Within']:.4f}")
    print(f"R-squared Between: {results['R-squared Between']:.4f}")
    print(f"R-squared Overall: {results['R-squared Overall']:.4f}")
    print(f"观测数: {results['N_obs']}")
    print(f"F-statistic: {results['F-statistic']:.4f}")
    print("\n系数估计:")
    for var, coef, tstat, pval in zip(
        results['Parameters'].index,
        results['Parameters'],
        results['t-stats'],
        results['P-values']
    ):
        print(f"{var:20} {coef:10.4f} (t={tstat:8.4f}) p={pval:.4f}")

# 将结果整理成DataFrame
summary_df = pd.DataFrame({
    'Y Variable': [],
    'R-squared': [],
    'R-squared Within': [],
    'R-squared Between': [],
    'R-squared Overall': [],
    'N_obs': [],
    'F-statistic': []
})

for y_var, results in regression_results.items():
    summary_df = pd.concat([summary_df, pd.DataFrame({
        'Y Variable': [y_var],
        'R-squared': [results['R-squared']],
        'R-squared Within': [results['R-squared Within']],
        'R-squared Between': [results['R-squared Between']],
        'R-squared Overall': [results['R-squared Overall']],
        'N_obs': [results['N_obs']],
        'F-statistic': [results['F-statistic']]
    })], ignore_index=True)

print("\n=== 回归结果汇总 ===")
print(summary_df)

# 导出到Excel
with pd.ExcelWriter('panel_regression_results.xlsx') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    # 每个回归的详细结果
    for y_var, results in regression_results.items():
        pd.DataFrame({
            'Coefficient': results['Parameters'],
            't-statistic': results['t-stats'],
            'P-value': results['P-values']
        }).to_excel(writer, sheet_name=f'{y_var[:30]}')



=== 因变量: close ===
R-squared: 0.0304
R-squared Within: 0.0122
R-squared Between: 0.0004
R-squared Overall: 0.0004
观测数: 1879
F-statistic: 9.1227

系数估计:
const                   61.5105 (t= 19.6830) p=0.0000
avg_sentiment           -0.3797 (t= -0.4514) p=0.6517
sentiment_std           -2.0278 (t= -0.4760) p=0.6342
comment_count           -0.0043 (t= -3.9541) p=0.0001
sentiment_consensus      2.0105 (t=  0.9990) p=0.3179
ma_5d                    8.4516 (t=  5.4240) p=0.0000
std_5d                   1.1867 (t=  0.6103) p=0.5417

=== 因变量: volume ===
R-squared: 0.2508
R-squared Within: 0.2443
R-squared Between: 0.1617
R-squared Overall: 0.1568
观测数: 1879
F-statistic: 97.4181

系数估计:
const                516830.5737 (t=  2.9066) p=0.0037
avg_sentiment        164660.5258 (t=  3.4408) p=0.0006
sentiment_std        -614567.6652 (t= -2.5353) p=0.0113
comment_count         1376.6221 (t= 22.2298) p=0.0000
sentiment_consensus  567453.1818 (t=  4.9555) p=0.0000
ma_5d                75887.1544 (t=  0.85